# FOODTRACK Z MAKARONEM 

W projekcie będzie rozpatrywana sytuacja dla małego przedsiębiorswa, która posiada foodtruck z makaronami 

In [1]:
import random

### CHARAKTERYSTYKA

Przedsiębiorstwo produkuje sosy w 4 smakach
    * Sos Bolosegne -> 10 zł
    * Sos Carbonara -> 12 zł
    * Sos Spinachi & Gorgonozola -> 11zł
    * Sos Brocoli -> 13 zł

Makarony są sprzedawane w dwóch rozmiarach
    * małym -> cena dania * 1 zł
    * dużym -> cena dania * 1.5 zł 

Makaron może posiadać dodatki np. Parmezan, Dodatkowe Oregano itp. Jaki kolwiek dodatek kosztuje 2 zł


Dodatkowo można zmienić rodzaj makaronu, podstawowy to spagetti
    * tagliatelle -> +2 zł
    * penne -> +1 zł 
    * farfelle -> +1 zł

In [2]:
PRICE_SAUCES = {
    "Sos Bolosegne" : 10,
    "Sos Carbonara" : 12,
    "Sos Spinachi & Gorgonozola" : 11,
    "Sos Brocoli" : 13
}

PRICE_SIZES = {
    "Mały" : 1,
    "DużY" : 1.5
}

PRICE_ADDITIONS = {
    "Brak dodatków" : 0,
    "Dodatki" : 2,
}

PRICE_PASTAS = {
    "Spagetti" : 0,
    "Tagliatelle" : 2,
    "Penne" : 1,
    "Farfelle" : 1
}

### POPYT

      Popyt na małe opakownie               Spagetti	Tagliatelle 	  Penne 	   Farfelle
    Sos Bolosegne 	                          10	        7	         11	        10
    Sos Bolosegne  z dodatkami	               8	        5	          9	         8
    Sos Carbonara	                            8	        5	          9	         8
    Sos Carbonara z dodatkami 	               6	        3	          7             6
    Sos Spinachi & Gorgonozola	               7	        4	          8             7
    Sos Spinachi & Gorgonozola z dodatkami	   5	        2	          6	         5
    Sos Brocoli	                              5	        2	          6	         5
    Sos Brocoli z dodatkami	                  3	        0              4	         3


Popyt na duże opakowanie o 1/2 mniejszy 

In [3]:
DEMAND = [[10, 7, 11, 10], [8, 5, 9, 8], [8, 5, 9, 8], [6,3, 7, 6], [7, 4, 8, 7], [5, 2, 6, 5], [5, 2, 6, 5],[3, 0, 4,3]]

print(*DEMAND, sep = '\n')

[10, 7, 11, 10]
[8, 5, 9, 8]
[8, 5, 9, 8]
[6, 3, 7, 6]
[7, 4, 8, 7]
[5, 2, 6, 5]
[5, 2, 6, 5]
[3, 0, 4, 3]


### KODOWANIE CECH 

Sos Bolosegne -> 00  
Sos Carbonara -> 01  
Sos Spinachi & Gorgonozola -> 10  
Sos Brocoli -> 11  

Brak dodatków -> 0  
Dodatki -> 1  

Mały rozmiar -> 0  
Duży rozmier -> 1   

Spagetti -> 00  
Tagliatelle -> 01  
Penne -> 10  
Farfelle -> 11  

NP => 100110 -> Spinachi & Gorgonzola bez dodatków średniego rozmiarów na mararonie Penne

Możliwości różnych marakonóœ jest = 4 * 2 * 2 * 4 = 64

In [4]:
SAUCES = {
    "00" : "Sos Bolosegne",
    "01" : "Sos Carbonara",
    "10" : "Sos Spinachi & Gorgonozola",
    "11" : "Sos Brocoli"
}

ADDITIONS = {
    "0": "Brak dodatków",
    "1" : "Dodatki"
}

SIZES = {
   "0" : "Mały",
   "1" : "DużY"
}

PASTAS = {
    "00" : "Spagetti",
    "01" : "Tagliatelle",
    "10" : "Penne",
    "11" : "Farfelle"
}

## FUNKCJA CELU 
Maxymalizacja zysków = popyt * cena

funkcja_celu = popyt_produktu * cena_produktu

In [5]:
def bin_to_int(obj):
    
    l = int(obj)
    num = 0
    i = 1
    
    while(l>0):
        num += l%10 * i
        l /= 10
        i *= 2
    
    return int(num)

In [6]:
def fitness(individual):
    
    sauce = individual[0] + individual[1]
    addition = individual[2]
    size = individual[3]
    pasta = individual[4] + individual[5]
    
    product_price = (PRICE_SAUCES[SAUCES[sauce]] + PRICE_ADDITIONS[ADDITIONS[addition]]
                     + PRICE_PASTAS[PASTAS[pasta]]) * PRICE_SIZES[SIZES[size]]  
    
    w = bin_to_int(sauce) * 2 + bin_to_int(addition)
    k = bin_to_int(pasta)

    if size == "1":
        demand_product = DEMAND[w][k] * 1/2
    else : 
        demand_product = DEMAND[w][k]
    
    return product_price * demand_product

## MUTACJA

In [7]:
def mutate_population(population):
    for x in population:
        for i in range(len(x)):
            if random.uniform(0, 1) < 0.05:
                x[i] = str(random.randint(0,1))
    return population

## TWORZENIE NOWEJ POPULACJI

In [8]:
def new_population(population, k=4):
    
    fitnesses = [fitness(x) for x in population]
    parts = [x / sum(fitnesses) for x in fitnesses]
    parts_sum = [sum(parts[:i+1]) for i in range(len(parts))]
    
    # Losowanie k nowych osobników
    result = []
    for _ in range(k):
        r = random.uniform(0, 1)
        
        # Szukanie najmniejszej większej od r
        for index in range(len(parts_sum)):
            if r < parts_sum[index]:
                result.append(population[index])
                break
                
    return result

## KRZYŻOWANIE

In [9]:
def crossing(a, b):
    assert len(a) == len(b)
    
    r = random.randint(1, len(a)-1)
    
    return a[:r] + b[r:]

In [10]:
def cross_population(population):
    result = population.copy()
    
    for i in range(len(result)):
        
        if random.randint(0, 1):
            
            b = random.choice(population[:i] + population[i+1:])
            result[i] = crossing(population[i], b)
    
    return result

## KOD GŁÓWNY

In [11]:
def best_individual(population):
    return max(population, key=fitness)

In [13]:
population = []

# Losujemy pierwsza populacje
for i in range(4):
    individual = []

    while len(individual) < 6:
        individual.append(str(random.randint(0,1)))

    population.append(individual)

best = best_individual(population)
for i in range(1000):
    population = new_population(population)
    population = cross_population(population)
    population = mutate_population(population)
    
    new_best = best_individual(population)
    # print(f'Best in population {i+1}: ({fitness(new_best)}) {new_best}')
    if fitness(new_best) > fitness(best):
        best = new_best

print(best)
print(fitness(best))
print(SAUCES[best[0] + best[1]], ADDITIONS[best[2]], SIZES[best[3]], PASTAS[best[4] + best[5]])

['0', '0', '0', '0', '1', '0']
121
Sos Bolosegne Brak dodatków Mały Penne
